In [2]:
def ipv4_to_32bit_int(ipv4):
    a, b, c, d = [int(x) for x in ipv4.split('.')]
    return (a << 24) + (b << 16) + (c << 8) + d


def interleave_bits(x, y):
    z = 0
    for i in range(16):
        z |= (x & 1 << i) << i | (y & 1 << i) << (i + 1)
    return z


def z_curve_mapping(ipv4):
    ip_int = ipv4_to_32bit_int(ipv4)
    x = ip_int & 0xAAAAAAAA
    y = ip_int & 0x55555555
    x = (x | (x >> 1)) & 0x33333333
    x = (x | (x >> 2)) & 0x0F0F0F0F
    x = (x | (x >> 4)) & 0x00FF00FF
    x = (x | (x >> 8)) & 0x0000FFFF
    y = (y | (y >> 1)) & 0x33333333
    y = (y | (y >> 2)) & 0x0F0F0F0F
    y = (y | (y >> 4)) & 0x00FF00FF
    y = (y | (y >> 8)) & 0x0000FFFF
    morton_number = interleave_bits(x, y)
    lat = (morton_number / (2 ** 32 - 1)) * 180 - 90
    lng = (morton_number % (2 ** 32 - 1)) * 360 / (2 ** 32 - 1) - 180
    return round(lat, 5), round(lng, 5)

def safe_z_curve_mapping(src_ip):
    try:
        lat, lon = z_curve_mapping(src_ip)
        return lat, lon
    except Exception as e:
        return None, None

In [19]:
import pandas as pd

df = pd.read_csv("requests.csv")

In [15]:
df['lat'], df['lon'] = zip(*df['dest_ip'].apply(lambda x: safe_z_curve_mapping(x)))

In [20]:
df = df.sort_values('timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'])
first_timestamp = df['timestamp'].iloc[0]
df['timestamp_delta'] = (df['timestamp'] - first_timestamp).dt.total_seconds()
df = df.drop(['src_ip','timestamp'], axis=1)
df.dropna()

,dest_ip,timestamp_delta
0,130.208.165.162,0.000
1,126.113.102.211,43.510
2,11.246.76.60,202.894
3,133.237.250.45,260.921
4,11.6.157.47,291.928
...,...,...
51673,118.242.100.101,5584.149
51674,3.173.82.147,5589.336
51675,0.34.188.54,5591.856
51676,207.10.85.210,5592.344


In [21]:
df.to_csv("requests_transformed.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51679 entries, 0 to 51678
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dest_ip          51678 non-null  object 
 1   lat              51678 non-null  float64
 2   lon              51678 non-null  float64
 3   timestamp_delta  51678 non-null  float64
dtypes: float64(3), object(1)
memory usage: 2.0+ MB
